# Spatial Planning Optimization for DRC 30 x 30 Target

This notebook executes the spatial planning optimization for recommending new areas to be protected in the DRC under the 30 x 30 initiative targets. The code is based on spatial input data that cen be retrieved using the pipeline in [XXX].

### Import Packages

In [1]:
from scripts import utils
import yaml

### Load Data

In [2]:
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

In [3]:
drc_df = utils.load_data(config, preprocess = True)

### Calculate the Multi-Objective Score

In [4]:
drc_df["objective_score"] = utils.multi_c_obj(drc_df, config)

### Visualize the Multi-Objective Score

In [5]:
utils.map_col(drc_df, "objective_score")

### Identify Candidate Planning Units for Protection 

In [6]:
drc_df = utils.iden_units(drc_df, config)

### Visualize Candidate Planning Units

In [7]:
utils.map_col(drc_df, "candidate", default_value=False)

### Set up Optimization Model 

Assumptions:
- Each planning unit can only be protected or not protected; there is no possibility for partial protection. 
- The goal of the optimization is to select planning units that maximize the multi-objective score.
- A bonus is added for selection planning units that are connected to existing or other selected planning units.
- The model maximzes the protection with a constraint to not go beyond 30% of the land. 

In [8]:
model, x = utils.setup_model(drc_df)

### Solve Spatial Optimization Model

In [9]:
drc_df = utils.solve_model(model, x, drc_df)

### Identify Selected Planning Units for Protection

In [ ]:
used_area = float((drc_df["area"] * drc_df["selected"]).sum())
print(f"Protected area: {used_area:.2f} ({used_area/cand_area_total:.2%})")

### Visualize Selected and Existing Protected Areas

In [ ]:
utils.map_col(drc_df, "selected")

In [ ]:
utils.map_col(drc_df, "protected")

In [ ]:
indices = drc_df.index[(drc_df["protected"] == 1) & (~drc_df["candidate"])]

In [ ]:
indices